<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/mnist_sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

In [4]:
new_mnist_trainset =  [ [[],[]] for i in range(10)]
new_mnist_testset  =  [ [[],[]] for i in range(10)]

for i in range(60000):
    for j in range(10):
        if mnist_trainset[i][1] == j:
            # image 
            new_mnist_trainset[j][0].append(mnist_trainset[i][0])  
            # label
            new_mnist_trainset[j][1].append(mnist_trainset[i][1])

for i in range(10000):
    for j in range(10):
        if mnist_testset[i][1] == j:
            # image 
            new_mnist_testset[j][0].append(mnist_testset[i][0])  
            # label
            new_mnist_testset[j][1].append(mnist_testset[i][1])

In [5]:
image_trainset = list()
label_trainset = list()

image_testset = list()
label_testset = list()

for i in range(10):
    image_trainset.append(new_mnist_trainset[i][0])
    label_trainset.append(new_mnist_trainset[i][1])

for i in range(10):
    image_testset.append(new_mnist_testset[i][0])
    label_testset.append(new_mnist_testset[i][1])

In [6]:
flattened_image_train = list()
flattened_label_train = list()

flattened_image_test = list()
flattened_label_test = list()

# flattening image 
for sublist in image_trainset:
    for val in sublist:
        flattened_image_train.append(val)

# flattening label
for sublist in label_trainset:
    for val in sublist:
        flattened_label_train.append(val)

# flattening image 
for sublist in image_testset:
    for val in sublist:
        flattened_image_test.append(val)

# flattening label
for sublist in label_testset:
    for val in sublist:
        flattened_label_test.append(val)

In [7]:
flattened_image_train = torch.stack(flattened_image_train)
flattened_label_train = torch.Tensor(flattened_label_train)
flattened_label_train = flattened_label_train.type(torch.LongTensor)

flattened_image_test = torch.stack(flattened_image_test)
flattened_label_test = torch.Tensor(flattened_label_test)
flattened_label_test = flattened_label_test.type(torch.LongTensor)

In [8]:
train_dataset = TensorDataset(flattened_image_train, flattened_label_train)
train_dataloader = DataLoader(train_dataset, batch_size=50)

test_dataset = TensorDataset(flattened_image_test, flattened_label_test)
test_dataloader = DataLoader(test_dataset, batch_size=50)

In [9]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

model = Model()

In [10]:
def get_activation():    
    def hook(module, input, output):
        model.layer_activations = output
    return hook
    
model.sigmoid.register_forward_hook(get_activation())

In [11]:
# adagrad 
optimizer  = torch.optim.Adagrad(model.parameters(), lr=0.1)

In [ ]:
f = open("sorted_mnist_adagrad.txt", "w")

criterion = torch.nn.CrossEntropyLoss()
no_epochs = 200
train_loss = list()
test_loss  = list()
test_acc   = list()
final_spareness = list()

# define activation list 

best_test_loss = 1

for epoch in range(no_epochs):
    total_train_loss = 0
    total_test_loss = 0

    hidden_layer_activation_list = list()

    # training
    # set up training mode 
    model.train()

    for itr, (image, label) in enumerate(train_dataloader):

        optimizer.zero_grad()

        pred = model(image)

        loss = criterion(pred, label)
        total_train_loss += loss.item()

        loss.backward()
        optimizer.step()

        # Print/Append activation of the hidden layer 
        # print(model.layer_activations.shape)
        hidden_layer_activation_list.append(model.layer_activations)
    
    # this conains activations for all epochs 
    final_spareness.append(hidden_layer_activation_list)

    total_train_loss = total_train_loss / (itr + 1)
    train_loss.append(total_train_loss)

    # testing 
    # change to evaluation mode 
    model.eval()
    total = 0
    for itr, (image, label) in enumerate(test_dataloader):

        pred = model(image)

        loss = criterion(pred, label)
        total_test_loss += loss.item()

        # we now need softmax because we are testing.
        pred = torch.nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1

    # caculate accuracy 
    accuracy = total / len(mnist_testset)

    # append accuracy here
    test_acc.append(accuracy)

    # append test loss here 
    total_test_loss = total_test_loss / (itr + 1)
    test_loss.append(total_test_loss)

    print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    if total_test_loss < best_test_loss:
        best_test_loss = total_test_loss
        print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
        torch.save(model.state_dict(), "model.dth")

for single_epoch_spareness in final_spareness:

    hidden_layer_activation_list = single_epoch_spareness
    hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
    layer_activations_list = torch.reshape(hidden_layer_activation_list, (60000, 256))

    layer_activations_list = torch.abs(layer_activations_list)  # modified 
    num_neurons = layer_activations_list.shape[1]
    population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
    mean_sparseness_per_epoch = torch.mean(population_sparseness)

    f = open("sorted_mnist_adagrad.txt", "a")
    f.write(str(mean_sparseness_per_epoch)+'\n')


Epoch: 1/200, Train Loss: 0.12857285, Test Loss: 7.54030974, Test Accuracy: 0.10090000

Epoch: 2/200, Train Loss: 0.09614688, Test Loss: 5.83546543, Test Accuracy: 0.10090000

Epoch: 3/200, Train Loss: 0.05961758, Test Loss: 4.87126793, Test Accuracy: 0.10680000

Epoch: 4/200, Train Loss: 0.04469364, Test Loss: 4.00259415, Test Accuracy: 0.17380000

Epoch: 5/200, Train Loss: 0.03706994, Test Loss: 3.68464195, Test Accuracy: 0.23810000

Epoch: 6/200, Train Loss: 0.03419592, Test Loss: 3.44937062, Test Accuracy: 0.27560000

Epoch: 7/200, Train Loss: 0.03199480, Test Loss: 3.33122518, Test Accuracy: 0.31660000

Epoch: 8/200, Train Loss: 0.03029283, Test Loss: 3.24316522, Test Accuracy: 0.33450000

Epoch: 9/200, Train Loss: 0.03035927, Test Loss: 3.11401232, Test Accuracy: 0.35900000

Epoch: 10/200, Train Loss: 0.02979577, Test Loss: 3.02490805, Test Accuracy: 0.36240000

Epoch: 11/200, Train Loss: 0.02894702, Test Loss: 2.87128641, Test Accuracy: 0.37450000

Epoch: 12/200, Train Loss: 0.

In [ ]:
f.close()

In [ ]:
!cp sorted_mnist_adagrad.txt /content/drive/MyDrive

In [ ]:
fig=plt.figure(figsize=(20, 10))
plt.plot(np.arange(1, no_epochs+1), sparseness_list, label="Sparseness", color='g')
plt.xlabel('Epochs')
plt.ylabel('Sparsity')
plt.ylim(0, 1)
plt.title("Sparsity Plot")
plt.legend(loc='best')
plt.show()

In [ ]:
fig=plt.figure(figsize=(20, 10))
plt.plot(np.arange(1, no_epochs+1), test_acc, label="Adagrad Test Accuracy", color='g')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.title("Accuracy Plot")
plt.legend(loc='best')
plt.show()